In [1]:
import copy
import glob
import time
import random
import regex as re
import numpy as np
import pandas as pd
import datetime as dt
import xgboost as xgb
import tensorflow as tf
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline
%run ../airquality/data/gen_daily_targets.py
%run ../airquality/data/prepare_data.py

In [4]:
test = pd.read_csv('../data/targets.csv')
stations = pd.read_csv('../data/stations.csv')
extra_features = pd.read_csv('/home/yc00032/Desktop/Just_Peanuts_II/BCNAirQualityDatathon/data/dataset_v1.csv', index_col=0).drop([
    'holiday', 'target', 
], axis=1)
rolling_mean_features = pd.read_csv('../data/data_roll_day_conc_obs.csv')
extra_features = extra_features[[c for c in extra_features.columns if not 'dist' in c]]

In [5]:
observations = pd.read_csv('../data/observations.csv', index_col=0)
models = pd.read_csv('../data/models.csv', index_col=0)
obs_and_mods = pd.merge(models, observations, how='left',
                    on=['station', 'day', 'time', 'datetime' ,'year'],
                    suffixes=('_mod', '')).sort_values('datetime', ascending=True)
obs_and_mods.to_csv('../data/obs_and_mod.csv')
obs_and_mods_cols = ['pred_0_days', 'pred_1_days', 'Concentration', 'target', 'day']
obs_and_mods.head()

,pred_0_days,pred_1_days,day,lon,lat,year,station,datetime,time,Concentration,target
0,31.81,31.59,2013-01-01,2.15,41.43,2013,ES1856A,2013-01-01 00:00:00,00:00:00,nan,nan
1,28.53,27.48,2013-01-01,2.12,41.39,2013,ES1992A,2013-01-01 00:00:00,00:00:00,nan,nan
2,28.53,27.48,2013-01-01,2.13,41.38,2013,ES1396A,2013-01-01 00:00:00,00:00:00,nan,nan
3,35.85,42.57,2013-01-01,2.20,41.40,2013,ES0691A,2013-01-01 00:00:00,00:00:00,nan,nan
4,38.79,34.11,2013-01-01,2.15,41.39,2013,ES1438A,2013-01-01 00:00:00,00:00:00,nan,nan


In [6]:
agg_types = ['mean', 'max', 'std']
tall_series = obs_and_mods.groupby(['day', 'station']).agg({
    'Concentration': agg_types
})['Concentration'].reset_index()
aggs = [tall_series.pivot(index='day', columns='station', values=agg) for agg in agg_types]
aggs = [df.rename(columns={c: c + '_' + agg for c in df.columns}) for df, agg in zip(aggs, agg_types)]
wide_series = pd.concat(aggs, axis=1)
wide_series.head()

station,ES0691A_mean,ES1396A_mean,ES1438A_mean,ES1480A_mean,ES1679A_mean,ES1856A_mean,ES1992A_mean,ES0691A_max,ES1396A_max,ES1438A_max,...,ES1679A_max,ES1856A_max,ES1992A_max,ES0691A_std,ES1396A_std,ES1438A_std,ES1480A_std,ES1679A_std,ES1856A_std,ES1992A_std
day,,,,,,,,,,,,,,,,,,,,,
2013-01-01,42.39,38.61,51.39,43.87,37.13,14.78,23.35,72.00,98.00,112.00,...,69.00,39.00,68.00,18.13,26.67,25.73,24.63,20.29,10.99,21.15
2013-01-02,52.50,47.58,63.71,54.50,48.75,32.08,43.29,87.00,86.00,119.00,...,76.00,77.00,85.00,16.61,24.72,26.48,19.18,16.91,25.43,26.53
2013-01-03,64.62,54.62,74.04,64.17,49.25,38.88,49.04,119.00,85.00,117.00,...,78.00,94.00,110.00,22.69,19.05,22.87,21.70,14.87,22.17,31.43
2013-01-04,54.12,44.30,44.50,89.13,61.92,22.18,39.04,80.00,113.00,123.00,...,108.00,60.00,99.00,16.58,28.35,32.46,45.41,25.83,16.98,28.20
2013-01-05,36.88,39.17,nan,89.00,64.00,12.25,33.54,60.00,114.00,nan,...,114.00,16.00,85.00,11.00,22.21,nan,21.63,17.04,2.56,17.75


In [7]:
test.station.unique()

array(['ES0691A', 'ES1396A', 'ES1438A', 'ES1480A', 'ES1679A', 'ES1856A',
       'ES1992A'], dtype=object)

## Cross Validation
Changes from `Jonas_5`:
* Add Jose's rolling mean features

In [8]:
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import mean_squared_error, make_scorer

lags = 25

for s in test.station.unique():
    data = wide_series[list(wide_series.reset_index().day < '2015-01-01')]
    to_lag = data[[c for c in data.columns if not c in ['{}_{}'.format(s, agg) for agg in agg_types]]]
    features = create_lagged_features(to_lag, lags)\
        .join(extra_features[extra_features.station == s].set_index('date'))\
        .join(rolling_mean_features[rolling_mean_features.station == s]
              .set_index('date').drop(['station', 'max_conc_obs'], axis=1))\
        .join(obs_and_mods[obs_and_mods.station == s][['Concentration', 'day']].groupby('day').max())
    X = features[[c for c in features.columns if not c in [
        'time', 'datetime', 'Concentration', 'target', 'station'
    ]]].values
    y = features['Concentration'].fillna(method='ffill').values

    params = pd.DataFrame({
        'eta': random.sample(list(np.linspace(0.01, 2, 201)), 200),
        'n_estimators': random.sample(list(range(1, 250)), 200),
        'reg_alpha': random.sample(list(np.linspace(0.01, 1, 201)), 200),
        'max_depth': random.sample(list(range(1, 35)), 25) * 8,
        'score': [np.nan] * 200
    })
    for i, r in params[['eta', 'n_estimators', 'reg_alpha', 'max_depth']].head(10).iterrows():
        kf = KFold(n_splits=3)
        metric = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            p = dict(r)
            p['n_estimators'] = int(p['n_estimators'])
            p['max_depth'] = int(p['max_depth'])
            model = xgb.XGBRegressor(**p)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            metric.append(
                mean_squared_error(y_test, y_pred)
            ) 

        params.loc[i, 'score'] = np.mean(metric)
        print(params.iloc[i])

    params.to_csv('../reports/cv_xgb_Jonas_6_{}_{}.csv'
                  .format(s, len(glob.glob('../reports/cv_xgb_Jonas_6_{}_*'.format(s)))))
    params.sort_values('score').head()

eta              1.38
max_depth        1.00
n_estimators   106.00
reg_alpha        0.11
score            0.96
Name: 0, dtype: float64


KeyboardInterrupt: 

In [9]:
all_frames = []
for s in test.station.unique():
    frames = [pd.read_csv(f, index_col=0) for f in glob.glob('../reports/cv_xgb_Jonas_6*{}*'.format(s))]
    try:
        frame = pd.concat(frames)
        frame['station'] = s
        all_frames.append(frame)
    except ValueError:
        pass
    
cv_results = pd.concat(all_frames)
cv_results = cv_results[cv_results['score'] > 0]
cv_results.sort_values(['score', 'station']).drop_duplicates('station')

,eta,max_depth,n_estimators,reg_alpha,score,station
6,0.51,18,70,0.19,18.54,ES0691A
9,1.72,2,222,0.81,24.07,ES1856A
2,1.06,6,40,0.59,30.97,ES1679A
3,0.56,3,85,0.69,34.96,ES1992A
4,1.64,4,107,0.06,55.69,ES1396A
9,0.46,6,227,0.69,70.77,ES1480A
9,0.78,22,189,0.09,97.92,ES1438A


In [10]:
test_params = []
for i, r in cv_results.sort_values(['score', 'station']).drop_duplicates('station').iterrows():
    p = dict(r)
    del p['score']
    del p['station']
    test_params.append((r['station'], p))
    
test_params

[('ES0691A',
  {'eta': 0.5075000000000001,
   'max_depth': 18,
   'n_estimators': 70,
   'reg_alpha': 0.1882}),
 ('ES1856A',
  {'eta': 1.7214, 'max_depth': 2, 'n_estimators': 222, 'reg_alpha': 0.8119}),
 ('ES1679A',
  {'eta': 1.0647, 'max_depth': 6, 'n_estimators': 40, 'reg_alpha': 0.5941}),
 ('ES1992A',
  {'eta': 0.55725,
   'max_depth': 3,
   'n_estimators': 85,
   'reg_alpha': 0.6930999999999999}),
 ('ES1396A',
  {'eta': 1.6418, 'max_depth': 4, 'n_estimators': 107, 'reg_alpha': 0.0595}),
 ('ES1480A',
  {'eta': 0.4577500000000001,
   'max_depth': 6,
   'n_estimators': 227,
   'reg_alpha': 0.6930999999999999}),
 ('ES1438A',
  {'eta': 0.77615,
   'max_depth': 22,
   'n_estimators': 189,
   'reg_alpha': 0.08919999999999999})]

In [ ]:
%%time
lags = 25
preds_by_station = []
edit_wide_series = copy.deepcopy(wide_series)

for d in test['date'].unique():
    data = edit_wide_series[list(pd.to_datetime(edit_wide_series.reset_index().day) <= d)]
    for s, p in test_params:
        to_lag = data[[c for c in data.columns if not c in ['{}_{}'.format(s, agg) for agg in agg_types]]]
        X = create_lagged_features(to_lag, lags)
        X_train = np.nan_to_num(X.iloc[:-1].values)
        for agg in agg_types:
            start = time.time()
            print('Predicting {} for station {} day {}'.format(agg, s, d))
            y_train = np.nan_to_num(data['{}_{}'.format(s, agg)][:-1])
            X_test = np.nan_to_num(X.iloc[-1].values.reshape(1, X.shape[1]))
            p['n_jobs'] = 4
            model = xgb.XGBRegressor(**p)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            edit_wide_series.loc[d, '{}_{}'.format(s, agg)] = y_pred
            print('Predicted {} in {:,.2f}s'.format(y_pred, time.time() - start))
            
edit_wide_series.to_csv('../data/jonas_6_output_wide_series.csv')

Predicting mean for station ES0691A day 2015-01-03
Predicted [ 55.21148682] in 1.96s
Predicting max for station ES0691A day 2015-01-03
Predicted [ 91.05612183] in 1.88s
Predicting std for station ES0691A day 2015-01-03
Predicted [ 18.69669724] in 1.81s
Predicting mean for station ES1856A day 2015-01-03
Predicted [ 34.20509338] in 0.88s
Predicting max for station ES1856A day 2015-01-03
Predicted [ 84.55183411] in 0.89s
Predicting std for station ES1856A day 2015-01-03
Predicted [ 21.84417725] in 0.89s
Predicting mean for station ES1679A day 2015-01-03
Predicted [ 45.00798416] in 0.50s
Predicting max for station ES1679A day 2015-01-03
Predicted [ 86.17668152] in 0.44s
Predicting std for station ES1679A day 2015-01-03
Predicted [ 17.88943291] in 0.43s
Predicting mean for station ES1992A day 2015-01-03
Predicted [ 31.74179268] in 0.52s
Predicting max for station ES1992A day 2015-01-03
Predicted [ 109.88838196] in 0.53s
Predicting std for station ES1992A day 2015-01-03
Predicted [ 21.495691

Predicted [ 81.00200653] in 0.83s
Predicting std for station ES1396A day 2015-01-15
Predicted [ 18.39378166] in 0.81s
Predicting mean for station ES1480A day 2015-01-15
Predicted [ 56.44588089] in 3.17s
Predicting max for station ES1480A day 2015-01-15
Predicted [ 100.89891815] in 3.01s
Predicting std for station ES1480A day 2015-01-15
Predicted [ 23.60967255] in 2.65s
Predicting mean for station ES1438A day 2015-01-15
Predicted [ 54.11917877] in 5.02s
Predicting max for station ES1438A day 2015-01-15
Predicted [ 112.05810547] in 5.92s
Predicting std for station ES1438A day 2015-01-15
Predicted [ 22.26839447] in 5.22s
Predicting mean for station ES0691A day 2015-01-18
Predicted [ 44.18230438] in 1.66s
Predicting max for station ES0691A day 2015-01-18
Predicted [ 82.06007385] in 1.72s
Predicting std for station ES0691A day 2015-01-18
Predicted [ 16.18363953] in 2.07s
Predicting mean for station ES1856A day 2015-01-18
Predicted [ 22.8753624] in 0.87s
Predicting max for station ES1856A da

Predicted [ 41.60560608] in 0.47s
Predicting max for station ES1679A day 2015-01-30
Predicted [ 74.06046295] in 0.47s
Predicting std for station ES1679A day 2015-01-30
Predicted [ 17.06516838] in 0.46s
Predicting mean for station ES1992A day 2015-01-30
Predicted [ 34.10922623] in 0.54s
Predicting max for station ES1992A day 2015-01-30
Predicted [ 76.17344666] in 0.55s
Predicting std for station ES1992A day 2015-01-30
Predicted [ 20.53112221] in 0.54s
Predicting mean for station ES1396A day 2015-01-30
Predicted [ 31.69244576] in 0.92s
Predicting max for station ES1396A day 2015-01-30
Predicted [ 59.83705521] in 0.93s
Predicting std for station ES1396A day 2015-01-30
Predicted [ 14.13265896] in 0.89s
Predicting mean for station ES1480A day 2015-01-30
Predicted [ 48.01377487] in 3.12s
Predicting max for station ES1480A day 2015-01-30
Predicted [ 90.26081848] in 3.26s
Predicting std for station ES1480A day 2015-01-30
Predicted [ 16.85664749] in 3.04s
Predicting mean for station ES1438A day

Predicted [ 19.77409554] in 5.85s
Predicting mean for station ES0691A day 2015-02-14
Predicted [ 46.55714035] in 1.92s
Predicting max for station ES0691A day 2015-02-14
Predicted [ 93.54322815] in 2.26s
Predicting std for station ES0691A day 2015-02-14
Predicted [ 21.52042007] in 2.43s
Predicting mean for station ES1856A day 2015-02-14
Predicted [ 46.3320694] in 0.98s
Predicting max for station ES1856A day 2015-02-14
Predicted [ 92.03109741] in 0.94s
Predicting std for station ES1856A day 2015-02-14
Predicted [ 20.44273567] in 0.95s
Predicting mean for station ES1679A day 2015-02-14
Predicted [ 52.86647797] in 0.54s
Predicting max for station ES1679A day 2015-02-14
Predicted [ 82.94111633] in 0.54s
Predicting std for station ES1679A day 2015-02-14
Predicted [ 17.53036118] in 0.54s
Predicting mean for station ES1992A day 2015-02-14
Predicted [ 43.43712616] in 0.53s
Predicting max for station ES1992A day 2015-02-14
Predicted [ 105.87594604] in 0.53s
Predicting std for station ES1992A day

Predicted [ 72.37844086] in 0.67s
Predicting std for station ES1396A day 2015-02-26
Predicted [ 16.5106411] in 0.65s
Predicting mean for station ES1480A day 2015-02-26
Predicted [ 60.36813736] in 2.43s
Predicting max for station ES1480A day 2015-02-26
Predicted [ 104.33222198] in 2.44s
Predicting std for station ES1480A day 2015-02-26
Predicted [ 22.25184822] in 2.26s
Predicting mean for station ES1438A day 2015-02-26
Predicted [ 59.99045944] in 4.04s
Predicting max for station ES1438A day 2015-02-26
Predicted [ 100.885849] in 4.34s
Predicting std for station ES1438A day 2015-02-26
Predicted [ 21.20335197] in 4.05s
Predicting mean for station ES0691A day 2015-03-01
Predicted [ 20.35991478] in 1.34s
Predicting max for station ES0691A day 2015-03-01
Predicted [ 40.77272034] in 1.39s
Predicting std for station ES0691A day 2015-03-01
Predicted [ 15.88288593] in 1.48s
Predicting mean for station ES1856A day 2015-03-01
Predicted [ 20.62637901] in 0.65s
Predicting max for station ES1856A day 

...

Predicted [ 56.30264664] in 1.10s
Predicting std for station ES1396A day 2015-08-13
Predicted [ 15.51788807] in 1.10s
Predicting mean for station ES1480A day 2015-08-13
Predicted [ 52.43405914] in 3.61s
Predicting max for station ES1480A day 2015-08-13
Predicted [ 84.50406647] in 3.46s
Predicting std for station ES1480A day 2015-08-13
Predicted [ 17.70823097] in 3.77s
Predicting mean for station ES1438A day 2015-08-13
Predicted [ 51.36484146] in 7.26s
Predicting max for station ES1438A day 2015-08-13
Predicted [ 94.33838654] in 7.01s
Predicting std for station ES1438A day 2015-08-13
Predicted [ 18.38977623] in 6.70s
Predicting mean for station ES0691A day 2015-08-16
Predicted [ 27.49160194] in 2.39s
Predicting max for station ES0691A day 2015-08-16
Predicted [ 55.14903641] in 2.45s
Predicting std for station ES0691A day 2015-08-16
Predicted [ 11.96143723] in 2.38s
Predicting mean for station ES1856A day 2015-08-16
Predicted [ 16.21110535] in 1.13s
Predicting max for station ES1856A day

Predicted [ 28.19548225] in 0.62s
Predicting max for station ES1679A day 2015-08-28
Predicted [ 57.17782593] in 0.56s
Predicting std for station ES1679A day 2015-08-28
Predicted [ 12.57221794] in 0.56s
Predicting mean for station ES1992A day 2015-08-28
Predicted [ 22.41067123] in 0.66s
Predicting max for station ES1992A day 2015-08-28
Predicted [ 50.97464752] in 0.66s
Predicting std for station ES1992A day 2015-08-28
Predicted [ 12.58875084] in 0.65s
Predicting mean for station ES1396A day 2015-08-28
Predicted [ 21.45107841] in 1.00s
Predicting max for station ES1396A day 2015-08-28
Predicted [ 40.39496231] in 0.99s
Predicting std for station ES1396A day 2015-08-28
Predicted [ 7.72042942] in 0.99s
Predicting mean for station ES1480A day 2015-08-28
Predicted [ 34.62267685] in 3.50s
Predicting max for station ES1480A day 2015-08-28
Predicted [ 64.29543304] in 3.50s
Predicting std for station ES1480A day 2015-08-28
Predicted [ 13.34628296] in 3.57s
Predicting mean for station ES1438A day 

Predicted [ 17.13024712] in 6.75s
Predicting mean for station ES0691A day 2015-09-12
Predicted [ 32.29547119] in 2.20s
Predicting max for station ES0691A day 2015-09-12
Predicted [ 50.23089218] in 2.23s
Predicting std for station ES0691A day 2015-09-12
Predicted [ 13.07282066] in 2.36s
Predicting mean for station ES1856A day 2015-09-12
Predicted [ 26.19290352] in 1.02s
Predicting max for station ES1856A day 2015-09-12
Predicted [ 55.16780853] in 1.02s
Predicting std for station ES1856A day 2015-09-12
Predicted [ 12.90499496] in 1.04s
Predicting mean for station ES1679A day 2015-09-12
Predicted [ 36.38762665] in 0.65s
Predicting max for station ES1679A day 2015-09-12
Predicted [ 68.01999664] in 0.56s
Predicting std for station ES1679A day 2015-09-12
Predicted [ 14.20110607] in 0.56s
Predicting mean for station ES1992A day 2015-09-12
Predicted [ 26.18615341] in 0.67s
Predicting max for station ES1992A day 2015-09-12
Predicted [ 63.78292465] in 0.66s
Predicting std for station ES1992A day

Predicted [ 50.88328171] in 1.12s
Predicting std for station ES1396A day 2015-09-24
Predicted [ 12.68350792] in 1.16s
Predicting mean for station ES1480A day 2015-09-24
Predicted [ 49.94949341] in 3.83s
Predicting max for station ES1480A day 2015-09-24
Predicted [ 80.41424561] in 3.60s
Predicting std for station ES1480A day 2015-09-24
Predicted [ 20.74570084] in 3.64s
Predicting mean for station ES1438A day 2015-09-24
Predicted [ 43.96772766] in 7.64s
Predicting max for station ES1438A day 2015-09-24
Predicted [ 79.38262177] in 8.00s
Predicting std for station ES1438A day 2015-09-24
Predicted [ 17.92534065] in 6.80s
Predicting mean for station ES0691A day 2015-09-27
Predicted [ 37.1162796] in 2.24s
Predicting max for station ES0691A day 2015-09-27
Predicted [ 66.54120636] in 2.34s
Predicting std for station ES0691A day 2015-09-27
Predicted [ 16.64943886] in 2.47s
Predicting mean for station ES1856A day 2015-09-27
Predicted [ 23.39351845] in 1.06s
Predicting max for station ES1856A day 

Predicted [ 39.35876465] in 0.65s
Predicting max for station ES1679A day 2015-10-09
Predicted [ 70.15602112] in 0.56s
Predicting std for station ES1679A day 2015-10-09
Predicted [ 15.88152218] in 0.56s
Predicting mean for station ES1992A day 2015-10-09
Predicted [ 35.0943985] in 0.70s
Predicting max for station ES1992A day 2015-10-09
Predicted [ 74.96468353] in 0.62s
Predicting std for station ES1992A day 2015-10-09
Predicted [ 19.39764023] in 0.69s
Predicting mean for station ES1396A day 2015-10-09
Predicted [ 29.99527931] in 1.14s
Predicting max for station ES1396A day 2015-10-09
Predicted [ 64.11971283] in 1.02s
Predicting std for station ES1396A day 2015-10-09
Predicted [ 14.68296337] in 1.16s
Predicting mean for station ES1480A day 2015-10-09
Predicted [ 52.8312645] in 4.23s
Predicting max for station ES1480A day 2015-10-09
Predicted [ 88.58033752] in 3.83s
Predicting std for station ES1480A day 2015-10-09
Predicted [ 18.98072624] in 3.82s
Predicting mean for station ES1438A day 2